# Project 02 - Due Friday, October 13 at 12pm

*Objectives*: Design a relational database schema for transaction processing. Use SQL to load the data into the database and retrieve information from the database.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries, shell commands, diagrams and markdown cells.  The notebook itself should be completely reproducible (using AWS EC2 instance based on the provided AMI) from start to finish; another person should be able to use the code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Attestation*: **Work in groups**.  At the end of your submitted notebook, identify the work each partner performed and attest that each contributed substantially to the work.

*Deadline*: Friday, October 13, 12pm.  One member of each group must submit your notebook to Blackboard; you should not submit it separately.

## Part 1 - Requirements

The city is going to launch a new bike sharing system. Your task is to design a relational database to support its operation. The bike sharing system works as follows:

1. Riders can buy annual, semiannual or monthly membership. We need to have a record of each member's name and address in order to send them a key. Keys can be used to check out bikes at any bike station. Members have the option to renew their memberships before they expire. A member becomes inactive when his or her membership expires, therefore we need to know when their memberships expire. 

1. Each bike station has a pubic name (i.e. "18th & C St NW"). Occasionally in response to public requests, stations can be renamed. Each bike station have been built with a number of docking points.

1. Each bike is either serviceable or disabled. If it is disabled, it is removed from the station for repair.

1. For simplicity's sake, each serviceable bike is either docked at a station or checked out by a member. If it is checked out by a member, the system needs to know who has checked it out and when.

Your database should minimally support the above requirements.

## Part 2 - Designing a Database Schema (35 points)

Based on what you read above, design a relational database schema consisting of *at least* 3 tables. Document your schema with a schema diagram.  There are several tools available for this, such as [Google Drawings](https://docs.google.com/drawings/) and Microsoft Visio. Make sure to include the schema diagram in your submission. Embed and display the diagram in your notebook. **Discuss your major design decisions.** Be aware that your schema is designed to support transaction processing and should be optimized for writes and data consistency.

We are looking for the following when grade this part of the project:

* All required attributes are recorded.
* In the schema diagram, all primary keys are underlined. Directed lines are drawn from the foreign keys to the primary keys.
* Your database schema should be designed to promite data consistency and integrity and reduce the duplication of data. Strive for 3NF.
* Your database schema should be designed to support the requirements described in Part 1, as well as problems in Part 4 and 5.
* Key design decisions are discussed and documented in Markdown cells.

See slide 23 of week-04 lecture for an example of schema diagram. 

---
* Let's look at the schema we created in google drawing

<img src=schema.jpg>

* discuss the major design decisions

Firstly we consider possible transactions, for example, the requirements described in Part 1, as well as problems in Part 4 and 5. Then we designed the relations(key, members, bike, station, goodbike, trace ), striving for 3NF.

In table key, every member has a key, so keyid should be the primary key of the table key. And the membership is to show the details  of key (annual, semiannual or monthly). The expiredate is to show when the key will become inactive. By this way, the table can record the transactions easily.

In table members, memberid is the ID of a member and the primary key, because members' names might be duplicate. Name and address is the personal information of a member. And keyid records the keys sent to the members. A member becomes inactive when his or her membership expires, so the status is to show whether the membership is active or not.

In table station, since the name of the station will change, we set the stationid ( i.e. the station number in the csv file) as the primary key. The publicname and capacity are the basic information of the station.

In table bike, each bike has its own id, so we set the bikeid as the primary key. Serviceable is to define whether the bike is on the service. 

In table goodbike. The bikeid is the bikeid which is serviceable in table bike. The bikestatus will show the bike is checked out or docked. If docked, the location will show where the bike is docked. 

In table trace, the bikeid is the bikeid which is checked out in table goodbike. The rider shows which member ride the bike. And the ridedate and startstation show when and where the bikes are checked out.


## Part 3 - ETL (30 points)
Please use markdown cells to **document major steps**. Use SQL queries and markdown cells to demonstrate that records are created or updated successfully.

### Problem 3.1
Examine the data located at https://s3.amazonaws.com/dmfa-2017/bike_stations.csv.
Create and connect to a new PostgreSQL database named `proj2`. Use SQL DDL to create your tables in it based on the database schema you designed in Part 2 and the data. Don't forget about the contraints (primary keys, foreign keys, proper attribute types, NOT NULL etc).

---
#### Create Repeatable Database
First, drop the database named `proj2` if it exists. It is necessary so that we can run this notebook repeatedly. If you get the error that says "database proj2 does not exist", that is fine. However if it complains that "There is 1 other session using the database", please restart the Kernel and try it again.

In [1]:
!dropdb -U student proj2

In [2]:
!createdb -U student proj2

In [2]:
%load_ext sql

* Use sql magic to connect to the database we just created. Use student as the user name. Password is not required here.

In [3]:
%sql postgresql://student@/proj2

'Connected: student@proj2'

In [5]:
%%sql
DROP TABLE IF EXISTS members,key,bike,station,goodbike,trace

Done.


[]

* Let's create tables. First we dropt the tables we created for further use --- running this notebook repeatedly. Second, we create the tables that include all relative columns required in Part 1.

In [6]:
%%sql
CREATE TABLE key(
    keyid CHAR(6) PRIMARY KEY, 
    membership VARCHAR(64) CONSTRAINT membership_check CHECK(membership in ('annual','semiannual','monthly')), 
    expiredate TIMESTAMP NOT NULL
);

Done.


[]

In [7]:
%%sql
CREATE TABLE members(
    memberid CHAR(5) PRIMARY KEY, 
    name VARCHAR(20) NOT NULL, 
    address VARCHAR(64) NOT NULL, 
    keyid CHAR(6) NOT NULL REFERENCES key(keyid),
    status VARCHAR(20) CONSTRAINT status_check CHECK(status in ('active','expired'))
);

Done.


[]

In [8]:
%%sql
CREATE TABLE station(
    stationid CHAR(5) PRIMARY KEY, 
    publicname VARCHAR(64) NOT NULL, 
    capacity INTEGER NOT NULL
);

Done.


[]

In [9]:
%%sql
CREATE TABLE bike(
    bikeid CHAR(6) PRIMARY KEY, 
    serviceable BOOLEAN
);

Done.


[]

In [10]:
%%sql
CREATE TABLE goodbike(
    bikeid CHAR(6)  PRIMARY KEY REFERENCES bike(bikeid), 
    bikestatus VARCHAR(64) CONSTRAINT bikestatus_check CHECK((bikestatus in ('docked','checkout')) OR (bikestatus IS NULL)), 
    location CHAR(5) REFERENCES station(stationid)
);

Done.


[]

In [11]:
%%sql
CREATE TABLE trace(
    bikeid CHAR(6) PRIMARY KEY REFERENCES goodbike(bikeid), 
    rider VARCHAR(64) REFERENCES members(memberid),
    ridedate TIMESTAMP,
    startstation CHAR(5) REFERENCES station(stationid)
);

Done.


[]

* Let's have a look at our tables' columns

In [12]:
%%sql
SELECT *
FROM members

0 rows affected.


memberid,name,address,keyid,status


In [13]:
%%sql
SELECT *
FROM key

0 rows affected.


keyid,membership,expiredate


In [14]:
%%sql
SELECT *
FROM station

0 rows affected.


stationid,publicname,capacity


In [15]:
%%sql
SELECT *
FROM bike

0 rows affected.


bikeid,serviceable


In [16]:
%%sql
SELECT *
FROM goodbike

0 rows affected.


bikeid,bikestatus,location


In [17]:
%%sql
SELECT *
FROM trace

0 rows affected.


bikeid,rider,ridedate,startstation


#### Summary
* From the tables above with no data embeded yet, they fit our designed schema. The <span style="color:#D4634B">member</span> table is use to record the members basic informations including they keys' type and their status. The <span style="color:#D4634B">key</span> table is depend on the membership type the buyer have chosen and the expire date of their membership.The <span style="color:#D4634B">bike</span> table is to record whether the bike is in use or not. The <span style="color:#D4634B">goodbike</span> table is for more information of the bike, which can locate the bike or detect the person who check it out. The <span style="color:#D4634B">station</span> table is to define the name of the station and the capacity of how many bikes could be docked at this location. The last <span style="color:#D4634B">trace</span> table is to record the check_out information at different start_station.   
*(The reason why we need the last form is to solve the Problem 4.3 better to display the information we needed when a bike has been checked out.)*

### Problem 3.2
Use the data located at https://s3.amazonaws.com/dmfa-2017/bike_stations.csv to populate your database. You may import the CSV file content into a temporary table in your database `proj2`. Just remember to remove it after you are done. 

Since the data doesn't include information about members, you will need to make up a few (4 or 5) members in your database. Some of their memberships have expired.

---
* First get the data from the site and load the data into our notebook.

In [18]:
!wget https://s3.amazonaws.com/dmfa-2017/bike_stations.csv

--2017-10-13 14:03:58--  https://s3.amazonaws.com/dmfa-2017/bike_stations.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.20.245
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.20.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169879 (166K) [text/csv]
Saving to: ‘bike_stations.csv.1’

bike_stations.csv.1 100%[===================>] 165.90K  --.-KB/s    in 0.006s  

2017-10-13 14:03:58 (27.2 MB/s) - ‘bike_stations.csv.1’ saved [169879/169879]



In [19]:
!csvsql --db postgresql://student@/proj2 --insert bike_stations.csv

* Let's look at the CSV table in the database.

In [20]:
%%sql
SELECT *
FROM bike_stations
LIMIT 5

5 rows affected.


Bike number,Bike serviceable,Station number,Station name,Station capacity
W00005,True,32011,Maple & Ritchie Ave,28
W00174,True,31246,M St & Pennsylvania Ave NW,10
W00196,True,31408,Takoma Metro,12
W00216,True,31309,Fessenden St & Wisconsin Ave NW,8
W00267,True,31125,15th & W St NW,24


* Load data to station table. We use insert instead of copy due to the fact we need to seperate the csv data to different tables.

In [21]:
%%sql
INSERT INTO station (stationid, publicname, capacity)
SELECT DISTINCT "Station number" as stationid, "Station name" as publicname, "Station capacity" as capacity
FROM bike_stations
WHERE "Station number" is NOT NULL

452 rows affected.


[]

* Load data to bike table.Load data to bike table.

In [22]:
%%sql
INSERT INTO bike (bikeid, serviceable)
SELECT DISTINCT "Bike number" as bikeid, "Bike serviceable" as serviceable
FROM bike_stations
WHERE "Bike number" is NOT NULL

4298 rows affected.


[]

In [23]:
%%sql
INSERT INTO goodbike (bikeid, location)
SELECT DISTINCT "Bike number" as bikeid, "Station number" as location
FROM bike_stations
WHERE (("Station number" is NOT NULL) and ("Bike number" is NOT NULL))

3823 rows affected.


[]

* Update bike status data into bike table by connecting the <span style="color:#0A9394">location</span> with the <span style="color:#0A9394">bike status</span>. 

In [24]:
%%sql
UPDATE goodbike SET bikestatus='docked'
WHERE (location is NOT NULL)

3823 rows affected.


[]

* Make sure the <span style="color:#0A9394">member</span> table and key table are clear.

In [25]:
%%sql
DELETE FROM members

0 rows affected.


[]

In [26]:
%%sql
DELETE FROM key

0 rows affected.


[]

* Load data into the <span style="color:#0A9394">member</span> table and <span style="color:#0A9394">key</span> table.

In [27]:
%%sql
INSERT INTO key (keyid, membership, expiredate)
VALUES('000001','annual','1/1/2020 23:59'),('000002','annual','1/1/2000 23:59'),
('000003','annual','1/1/2020 23:59'),('000004','monthly','1/1/2000 23:59')

4 rows affected.


[]

In [28]:
%%sql
INSERT INTO members (memberid, name, address, keyid, status)
VALUES('00001','Zoey','CP','000001','active'),('00002','Jacky','CP','000002','expired'),
('00003','Leo','CP','000003','active'),('00004','Charlene','CP','000004','expired')

4 rows affected.


[]

* **Delete** the temporary CSV table.

In [29]:
%%sql
DROP TABLE IF EXISTS bike_stations

Done.


[]

## Part 4 - Operations (15 points)
Use SQL queries and markdown cells to demonstrate that records are created or updated successfully.

### Problem 4.1
The bike 'W20131' is having mechanical problems and needs to be repaired off-site. Use SQL UPDATE statements to record the fact that this bike is no longer serviceable and has been removed from the station where it was docked at.

---
#### Update different information into our database
* Let's find the information about the bike and **update** the record by changing the bike serviceable column to Fales and delete from the goodbike table.

In [30]:
%%sql
SELECT *
FROM bike
LEFT OUTER JOIN goodbike on bike.bikeid=goodbike.bikeid
WHERE bike.bikeid='W20131';

1 rows affected.


bikeid,serviceable,bikeid_1,bikestatus,location
W20131,True,W20131,docked,31060


In [31]:
%%sql
UPDATE bike SET serviceable=False
  WHERE bikeid='W20131';
DELETE FROM goodbike
  WHERE bikeid='W20131';

1 rows affected.
1 rows affected.


[]

* Check the bike information again for the changes we made.

In [32]:
%%sql
SELECT *
FROM bike
WHERE bikeid='W20131';

1 rows affected.


bikeid,serviceable
W20131,False


In [33]:
%%sql
SELECT *
FROM goodbike
WHERE bikeid='W20131';

0 rows affected.


bikeid,bikestatus,location


### Problem 4.2
The bike station at Adams Mill & Columbia Rd NW has been expanded. Use SQL UPDATE statements to change the name of the bike station from "Adams Mill & Columbia Rd NW" to "18th & Columbia Rd NW" and increase its capacity by 10.

* Let's find the information about the station and update its name and capacity.

In [34]:
%%sql
SELECT *
FROM station
WHERE publicname='Adams Mill & Columbia Rd NW'

1 rows affected.


stationid,publicname,capacity
31104,Adams Mill & Columbia Rd NW,4


In [35]:
%%sql
UPDATE station SET capacity=capacity+10
WHERE publicname='Adams Mill & Columbia Rd NW';
UPDATE station SET publicname='18th & Columbia Rd NW' 
WHERE publicname='Adams Mill & Columbia Rd NW';

1 rows affected.
1 rows affected.


[]

* Check the station information again for the changes we made.

In [36]:
%%sql
SELECT *
FROM station
WHERE (stationid='31104' or publicname='Adams Mill & Columbia Rd NW');

1 rows affected.


stationid,publicname,capacity
31104,18th & Columbia Rd NW,14


### Problem 4.3
Use SQL UPDATE and/or INSERT statements to record the transaction that an active member checked out the bike 'W00005' from station 'Maple & Ritchie Ave' at '2017-10-01 08:46:00'.

* First update the table that one of the riders recorded to check out the bike on certain condition.

In [37]:
%%sql
SELECT *
FROM goodbike
LEFT OUTER JOIN station ON goodbike.location=station.stationid
WHERE bikeid='W00005'

1 rows affected.


bikeid,bikestatus,location,stationid,publicname,capacity
W00005,docked,32011,32011,Maple & Ritchie Ave,28


* Then Check the bike information we updated showing the information we needed.

In [38]:
%%sql
INSERT INTO trace (bikeid,rider,ridedate,startstation)
VALUES ('W00005','00001','2017-10-01 08:46:00','32011');
UPDATE goodbike SET location=Null,bikestatus='checkout'
WHERE bikeid='W00005';

1 rows affected.
1 rows affected.


[]

In [39]:
%%sql
UPDATE goodbike SET bikestatus='checkout', location=Null
WHERE bikeid='W00005'

1 rows affected.


[]

* Check the bike information.

In [8]:
%%sql
SELECT bike.bikeid, goodbike.bikestatus, trace.rider, members.name, members.status, trace.ridedate,startstation,publicname
FROM bike
LEFT OUTER JOIN goodbike on bike.bikeid=goodbike.bikeid
LEFT OUTER JOIN trace ON goodbike.bikeid=trace.bikeid
LEFT OUTER JOIN members ON members.memberid=trace.rider
LEFT OUTER JOIN key ON key.keyid=members.keyid
LEFT OUTER JOIN station ON station.stationid=trace.startstation
WHERE bike.bikeid='W00005'

1 rows affected.


bikeid,bikestatus,rider,name,status,ridedate,startstation,publicname
W00005,checkout,00001,Zoey,active,2017-10-01 08:46:00,32011,Maple & Ritchie Ave


#### Summary
* This Part 4 helps us to understand how we can manupulate the database by using Update and Insert to record the changes.

## Part 5 - Queries (20 points)

Use SQL SELECT statements to answer the following questions. Use markdown cells to explain your SQL statements and results if needed.

### Problem 5.1
For all members, display their name, membership type (annual, semiannual or monthly, in readable text), membership status (active or expired, boolean value is fine) and the date when the membership expires.

---
* **Select** the columns in the <span style="color:#0A9394">member</span> tables that display the information we needed.   
  
* **Result** showing that two person are in active status and the other two have expired membership. Three of the member have annual pass. 

In [41]:
%%sql
SELECT name, membership, status, expiredate
FROM members
JOIN key ON members.keyid=key.keyid

4 rows affected.


name,membership,status,expiredate
Zoey,annual,active,2020-01-01 23:59:00
Jacky,annual,expired,2000-01-01 23:59:00
Leo,annual,active,2020-01-01 23:59:00
Charlene,monthly,expired,2000-01-01 23:59:00


### Problem 5.2
Display the first 10 bikes base on the bike number in ascending order. For each bike, list its status (serviceable or disabled, boolean value is fine), the station name where it is docked at if there is any, the member if the bike is currently checked out, and when it was checked out. Hint: you should see one disabled bike and one checked out bike.

* In order to display all the information required in this problem, we used ** Left Outer Join** method to attach the different tables together to show the information we needed and select the first 10 bikes in ASC order. The **Result**
is showing below. 

In [42]:
%%sql
SELECT bike.bikeid, serviceable, bikestatus, location, station.publicname, rider, ridedate
FROM bike
LEFT OUTER JOIN goodbike ON bike.bikeid=goodbike.bikeid
LEFT OUTER JOIN trace ON trace.bikeid=goodbike.bikeid
LEFT OUTER JOIN station ON station.stationid=goodbike.location
ORDER BY bike.bikeid ASC
LIMIT 10

10 rows affected.


bikeid,serviceable,bikestatus,location,publicname,rider,ridedate
W00001,True,docked,31301,Ward Circle / American University,None,None
W00005,True,checkout,None,None,00001,2017-10-01 08:46:00
W00007,True,docked,31508,Gallaudet / 8th St & Florida Ave NE,None,None
W00009,True,docked,31912,Mount Vernon Ave & Bruce St,None,None
W00010,True,docked,31249,Jefferson Memorial,None,None
W00011,False,None,None,None,None,None
W00012,True,docked,31902,Wilson Blvd & N Illinois St,None,None
W00013,True,docked,31914,Lee Center,None,None
W00014,True,docked,31902,Wilson Blvd & N Illinois St,None,None
W00015,True,docked,31043,Saint Asaph St & Pendleton St,None,None


### Problem 5.3
Display the top 10 stations which currently have the most bikes docked. For each station, list the station name and the total number of bikes docked there.

* **Steps:** To find the most bikes docked, we need to count the bikeid in different stations first, and then we need to join the station table with  the new 'goodbike' table to showing the result that how many bikes are docked in the different station. 
  
* The **result** as follow denote that top ten stations have the most bikes docked from this data.

In [43]:
%%sql
SELECT publicname, COUNT(bikeid) as "total number of bikes docked"
FROM goodbike
JOIN station ON goodbike.location=station.stationid
WHERE location IS NOT NULL
GROUP BY publicname
ORDER BY COUNT(bikeid) DESC
LIMIT 10;

10 rows affected.


publicname,total number of bikes docked
Columbus Circle / Union Station,49
14th & V St NW,41
2nd & G St NE,36
13th & D St NE,33
12th & U St NW,32
11th & Kenyon St NW,29
1st & N St SE,28
6th & H St NE,27
4th & M St SW,27
13th & H St NE,27


### Problem 5.4
Display the first 10 stations based on their names in ascending order. For each station, list station name and the number of available docking points accepting bike returns. Verify the numbers are correct.

* **Steps:** To calculate the available docking points, we need to use the station's capacitu minus the docked bike number and then documenated them into a new column.
  
* The **result** as follow displays that top ten stations in names ascending order with the available docking points accepting bike returns.

In [44]:
%%sql
SELECT publicname,capacity,COUNT(bikeid) AS bikenumber,(capacity-COUNT(bikeid)) AS available
FROM (goodbike JOIN station ON goodbike.location = station.stationid)
WHERE location is NOT NULL
GROUP BY publicname,capacity
ORDER BY publicname ASC
LIMIT 10

10 rows affected.


publicname,capacity,bikenumber,available
10th & E St NW,17,15,2
10th & Florida Ave NW,21,16,5
10th & G St NW,14,6,8
10th & K St NW,14,8,6
10th & Monroe St NE,3,1,2
10th St & Constitution Ave NW,13,8,5
10th St & L'Enfant Plaza SW,8,1,7
10th & U St NW,17,15,2
11th & F St NW,8,6,2
11th & H St NE,23,20,3


### <table><tr><td bgcolor=#D1EEEE><font face="Cursive"><font color="#660000"><font size="10">Thank you for reviewing this notebook.</font></span></td></tr></table>

Group Work: Yuying Zhou (Main coding), Minxuan Chen (Main Coding), Huilin Cai(Decision Design), Yuxuan Liu(Main comments). Everybody has brought their efforts in every parts together.